# Bike Share Rebalancing Minimizer - Data Aggregation

### ACKNOWLEDGEMENTS:

Many thanks to: [dailyLi](https://github.com/dailyLi/toronto_bike_share/blob/main/Preprocessing.ipynb), [Pro Geomatics](https://www.youtube.com/watch?v=wIWzjMiYjeY), [user2856](https://gis.stackexchange.com/questions/384581/raster-to-geopandas)

### OBJECTIVE:

The purpose of this section of the project is to collect data from various sources to make a data frame that contains the following information:

| Column | Data Type | Description |
| --- | --- | --- |
| station_id | int16 | the station's unique identifier |
| lat | float64 | the station's latitude |
| lon | float64 | the station's longitude |
| elevation | float64 | the station's elevation |
| start_count | int32 | number of rides started at the station |
| end_count | int32 | number of rides ended at the station |
| use_count | int32 | total station usage (start_count + end_count) |
| end_ratio | float64 | ratio of the station's use that is for ending rides (end_count / use_count) |


### DATA SOURCES:
   * [Bike Share Ride Data - Toronto Open Data](https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/)
   * [Bike Share Station Lat/Long - Toronto Open Data](https://open.toronto.ca/dataset/bike-share-toronto/)
   * [Elevation Data - Canada Open Government](https://open.canada.ca/data/en/dataset/f5c4e4af-ddc5-4f54-a70b-8390e4a4268e/resource/63ef74a5-4c9d-42b8-a0a7-68730a350759)
   
### STEPS:

   * [1. Get Ride Data](#1)
   * [2. Aggregate Ride Data](#2)
   * [3. Add Long/Lat Data](#3)
   * [4. Add Elevation Data](#4)
   * [5. Export Data](#5)

### 1. GET RIDE DATA <a class="anchor" id="1"></a>

In [25]:
# Import libraries
import pandas as pd
import numpy as np
from collections import Counter
import glob
import json
import requests
import pyproj
import rasterio as rio

In [26]:
# For each .csv file in the folder "data", read the specified columns and concatenate everything into one data frame
# Note: .csv files from Toronto Open Data contain an error double space in "Trip  Duration"
ride_cols = ['Trip Id', 'Start Station Id', 'Start Time', 'End Station Id', 'End Time', 'User Type', 'Trip  Duration', 'Bike Id']
data = pd.concat(map(lambda x: pd.read_csv(x, usecols=ride_cols), glob.glob('data/*.csv')), ignore_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4620469 entries, 0 to 4620468
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Trip Id           int64  
 1   Trip  Duration    int64  
 2   Start Station Id  int64  
 3   Start Time        object 
 4   End Station Id    float64
 5   End Time          object 
 6   Bike Id           int64  
 7   User Type         object 
dtypes: float64(1), int64(4), object(3)
memory usage: 282.0+ MB


In [27]:
# Function to check data for 0 and null values
def checkNa0(df):
    return pd.concat(
            [pd.DataFrame(df.isna().sum(), columns=['is_na']),
            pd.DataFrame(df[(data == 0)].count(), columns=['is_0'])],
            axis=1
             )

# Check data for 0 and null values
checkNa0(data)

,is_na,is_0
Trip Id,0,0
Trip Duration,0,4591
Start Station Id,0,0
Start Time,0,0
End Station Id,1279,0
End Time,0,0
Bike Id,0,0
User Type,0,0


In [28]:
# Drop NA "End Station Id" values
data = data.dropna()

# Drop rows where "Trip  Duration" is 0
data = data.loc[data['Trip  Duration'] != 0]

In [29]:
# Check for abnormalities in "User Type" (i.e. is there a category--such as "Employee"--that is unwanted?)
data['User Type'].unique()

array(['Casual Member', 'Annual Member'], dtype=object)

In [30]:
# Drop unneeded columns, which were only added to check for abnormalities in the data
data = data.drop(['Start Time', 'End Time', 'Trip  Duration', 'Bike Id', 'User Type'], axis=1)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4615838 entries, 0 to 4620468
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Trip Id           int64  
 1   Start Station Id  int64  
 2   End Station Id    float64
dtypes: float64(1), int64(2)
memory usage: 140.9 MB


### 2. AGGREGATE RIDE DATA <a class="anchor" id="2"></a>

In [32]:
# Count the number of times a station ID appears in the data as "start station" and "end station", then merge those two new dfs together
station_data = pd.merge(
    pd.DataFrame.from_dict(Counter(data['Start Station Id']), orient='index', columns=['start_count']).reset_index(),
    pd.DataFrame.from_dict(Counter(data['End Station Id']), orient='index', columns=['end_count']).reset_index(),
    how='outer',
    on='index')

# Check data for 0 and null values
checkNa0(station_data)

,is_na,is_0
index,0,0
start_count,2,0
end_count,0,0


In [33]:
station_data.query('start_count.isnull()')

,index,start_count,end_count
682,7690.0,NaN,17
683,7756.0,NaN,1


In [34]:
# Change null values to 0
station_data.loc[682:683, 'start_count'] = 0

In [35]:
# Make a new column that is the sum of start_count and end_count
station_data['use_count'] = station_data['start_count'] + station_data['end_count']

# Make a new column that is the ratio of end_count / use_count
station_data['end_ratio'] = station_data['end_count'] / station_data['use_count']

# Rename 'index' column
station_data.rename(columns={'index': 'station_id'}, inplace=True)

station_data.head()

,station_id,start_count,end_count,use_count,end_ratio
0,7334.0,6584.0,7230,13814.0,0.523382
1,7443.0,8167.0,7332,15499.0,0.473063
2,7399.0,23747.0,25184,48931.0,0.514684
3,7524.0,16262.0,16042,32304.0,0.496595
4,7365.0,2939.0,3208,6147.0,0.521881


### 3. ADD STATION LONG/LAT <a class="anchor" id="3"></a>

In [36]:
# Request JSON bike station data, and put it into a data frame
json_request = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information').content
lat_long_data = pd.DataFrame(json.loads(json_request)['data']['stations'])[['station_id', 'lat', 'lon']].astype({
    'station_id': 'int',
})

# Add latitude and longitude of start station and end station to data frame, remove unneeded columns
station_data = pd.merge(station_data, lat_long_data, how='left', on='station_id')

station_data.head()

,station_id,start_count,end_count,use_count,end_ratio,lat,lon
0,7334.0,6584.0,7230,13814.0,0.523382,43.646218,-79.385505
1,7443.0,8167.0,7332,15499.0,0.473063,43.657477,-79.373446
2,7399.0,23747.0,25184,48931.0,0.514684,43.644220,-79.369270
3,7524.0,16262.0,16042,32304.0,0.496595,43.642385,-79.424028
4,7365.0,2939.0,3208,6147.0,0.521881,43.666325,-79.301400


In [37]:
# Check data for 0 and null values
checkNa0(station_data)

,is_na,is_0
station_id,0,0
start_count,0,0
end_count,0,0
use_count,0,0
end_ratio,0,0
lat,40,0
lon,40,0


In [38]:
# Remove that have missing latitude data
station_data = station_data[(station_data['lat'].notna()) & (station_data['lon'].notna())]

In [39]:
# Reset index
station_data = station_data.sort_values('station_id').reset_index(drop=True)

### 4. ADD ELEVATION DATA <a class="anchor" id="4"></a>

In [40]:
# Make a transformer to change from long/lat to utm. CRS codes were found on https://epsg.io/
lonlat_to_utm = pyproj.Transformer.from_crs(4326, 32617)

# Make a data frame of UTM coordinates for each stations
station_utm = pd.DataFrame(lonlat_to_utm.transform(station_data['lat'],station_data['lon'])).transpose()
station_utm.columns = ['utm_east', 'utm_north']

# Round UTM coordinates to the nearest 5, since that's the resolution of the elevation data
def roundTo5(x, base=5):
    return base * round(x/base)

station_utm = station_utm.apply(roundTo5)

# Update station_data and change dtypes
station_data = pd.concat([station_data, station_utm], axis=1)

station_data.head()

,station_id,start_count,end_count,use_count,end_ratio,lat,lon,utm_east,utm_north
0,7000.0,22889.0,22449,45338.0,0.495148,43.639832,-79.395954,629380.0,4833120.0
1,7001.0,15523.0,15545,31068.0,0.500354,43.664964,-79.383550,630325.0,4835930.0
2,7002.0,21183.0,20835,42018.0,0.495859,43.667333,-79.399429,629040.0,4836170.0
3,7003.0,9653.0,8418,18071.0,0.465829,43.667158,-79.402761,628770.0,4836145.0
4,7004.0,6826.0,6390,13216.0,0.483505,43.656518,-79.389099,629895.0,4834985.0


In [41]:
# Make array of file locations of elevation data .tif files, originally downloaded from:
# https://open.canada.ca/data/en/dataset/f5c4e4af-ddc5-4f54-a70b-8390e4a4268e/resource/63ef74a5-4c9d-42b8-a0a7-68730a350759?inner_span=True
elevation_data = [
    './SW/dm_600481.tif', './SW/dm_600483.tif', './SW/dm_600485.tif',
    './SW/dm_620481.tif', './SW/dm_620483.tif', './SW/dm_620485.tif',
    './SW/dm_640483.tif', './SW/dm_640485.tif'
]

# Loop to change each filename in the array "elevation_data" to the content of that file as a data frame
for i, file in enumerate(elevation_data):

    # Import elevation data from .tif files to data frame
    with rio.Env():
        with rio.open(file) as src:

            # Create arrays for x and y axis of the raster using UTM EPSG:26917 values (coordinate system of source files)
            xmin, ymax = np.around(src.xy(0.00, 0.00), 9)
            xmax, ymin = np.around(src.xy(src.height-1, src.width-1), 9)
            x = np.linspace(xmin, xmax, src.width)
            y = np.linspace(ymax, ymin, src.height)  # max -> min so coords are top -> bottom

            # Create 2D arrays
            xs, ys = np.meshgrid(x, y) # I don't understand why this returns an array shape (4401,) ...shouldn't it be 2D?
            zs = src.read(1)

            # Apply NoData mask - i.e. only get rows for coordinates with elevation data
            mask = src.read_masks(1) > 0
            xs, ys, zs = xs[mask], ys[mask], zs[mask]

    df = {"utm_east": pd.Series(xs.ravel()),
            "utm_north": pd.Series(ys.ravel()),
            "elevation": pd.Series(zs.ravel())}

    elevation_data[i] = pd.DataFrame(data=df)

    # Remove elevation data for areas that have no bike stations, to save memory
    mask1 = elevation_data[i]['utm_east'] >= 617605
    mask2 = elevation_data[i]['utm_east'] <= 650980
    mask3 = elevation_data[i]['utm_north'] >= 4827155
    mask4 = elevation_data[i]['utm_north'] <= 4850075
    elevation_data[i] = elevation_data[i].loc[(mask1) & (mask2) & (mask3) & (mask4),:]

    # Change utm coordinates to int32, to save memory
    elevation_data[i]['utm_east'] = elevation_data[i]['utm_east'].astype('int32')
    elevation_data[i]['utm_north'] = elevation_data[i]['utm_north'].astype('int32')

# Combine elevation data into one data frame
elevation_data = pd.concat(elevation_data)

# Drop duplicate entries (I don't understand why there are duplicate entries)
elevation_data = elevation_data.drop_duplicates()

# Add elevation to station_data
station_data = pd.merge(station_data, elevation_data, how='left', on=['utm_east', 'utm_north'])

station_data.head()

,station_id,start_count,end_count,use_count,end_ratio,lat,lon,utm_east,utm_north,elevation
0,7000.0,22889.0,22449,45338.0,0.495148,43.639832,-79.395954,629380.0,4833120.0,83.367996
1,7001.0,15523.0,15545,31068.0,0.500354,43.664964,-79.383550,630325.0,4835930.0,107.094704
2,7002.0,21183.0,20835,42018.0,0.495859,43.667333,-79.399429,629040.0,4836170.0,114.094002
3,7003.0,9653.0,8418,18071.0,0.465829,43.667158,-79.402761,628770.0,4836145.0,114.199348
4,7004.0,6826.0,6390,13216.0,0.483505,43.656518,-79.389099,629895.0,4834985.0,95.365051


In [42]:
# Check data for 0 and null values
checkNa0(station_data)

,is_na,is_0
station_id,0,0
start_count,0,0
end_count,0,0
use_count,0,0
end_ratio,0,0
lat,0,0
lon,0,0
utm_east,0,0
utm_north,0,0
elevation,1,0


In [43]:
# Find the row with the missing data
station_data[station_data['elevation'].isnull()]

,station_id,start_count,end_count,use_count,end_ratio,lat,lon,utm_east,utm_north,elevation
524,7623.0,2967.0,2999,5966.0,0.502682,43.60299,-79.492687,621650.0,4828885.0,NaN


In [44]:
# Since there is only one row with missing data, the missing or value will be added by looking at a close value:
elevation_data[(elevation_data['utm_east'] < 621650) & (elevation_data['utm_north'] == 4828885)
               ].sort_values(['utm_east', 'utm_north'], ascending=False)

,utm_east,utm_north,elevation
155363,620995,4828885,83.099304
155362,620990,4828885,83.094482
155361,620985,4828885,83.048294
155360,620980,4828885,83.000679
155359,620975,4828885,82.984505
...,...,...,...
895748,617625,4828885,102.391586
895747,617620,4828885,102.487068
895746,617615,4828885,102.525604
895745,617610,4828885,102.556213


In [45]:
# Add the value to station 7623
station_data.loc[station_data['station_id'] == 7623, 'elevation'] = 83.099304

# Drop UTM columns, which were used for adding elevation data
station_data = station_data.drop(columns=['utm_east', 'utm_north'])

### 4. EXPORT DATA <a class="anchor" id="4"></a>

In [46]:
# Check max values in each column, to see what dtype each column can be cast down to
station_data.max()

station_id      7760.000000
start_count    40380.000000
end_count      45788.000000
use_count      86168.000000
end_ratio          1.000000
lat               43.788319
lon              -79.123505
elevation        200.851028
dtype: float64

In [47]:
# Change data types
station_data = station_data.astype({
    'station_id': 'int16',
    'start_count': 'int32',
    'end_count': 'int32',
    'use_count': 'int32'})

station_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 644 entries, 0 to 643
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station_id   644 non-null    int16  
 1   start_count  644 non-null    int32  
 2   end_count    644 non-null    int32  
 3   use_count    644 non-null    int32  
 4   end_ratio    644 non-null    float64
 5   lat          644 non-null    float64
 6   lon          644 non-null    float64
 7   elevation    644 non-null    float64
dtypes: float64(4), int16(1), int32(3)
memory usage: 50.1 KB


In [48]:
# Export ride data
station_data.to_parquet('station_data.parquet', engine='pyarrow')